In [ ]:
import os
import mujoco
import math
import numpy
import mediapy as media
import matplotlib.pyplot as plt
from idealab_tools.kinematics.kinematics import Quaternion


# import distutils.util
# import subprocess

In [ ]:
width = 640
height = 480
timestep = 1e-3

In [ ]:
class SG90Data(object):


    Vnom = 6
    G = 55.5
    t_stall = 15/100/G
    i_stall = .6
    R = Vnom/i_stall
    i_nl = .2
    w_nl = .66*1000*math.pi/180*G

    kt = t_stall/ i_stall
    # kv= Vnom/w_nl
    ke = kt

    b = kt*i_nl/w_nl

    ts = 1e-4

    V_supply = 5
    kp = 8.896e+00
    b_act = 1.404e-06

    @classmethod
    def calc_servo_torque(cls,desired,q,w):
        kp=cls.kp
        b=cls.b_act
        G=cls.G
        R=cls.R
        ke=cls.ke
        kt=cls.kt
        V_supply = cls.V_supply
        V,i = cls.calc_servo_command(desired,q,w)
        torque = (kt*(V-(ke)*w*G)/R-b*w*G)*G        
        return torque,V,i

    @classmethod
    def calc_servo_command(cls,desired,q,w):
        kp=cls.kp
        V_supply = cls.V_supply
        ke=cls.ke
        G=cls.G
        R=cls.R

        error = desired - q
        V = kp*error
        if V>V_supply: V=V_supply
        if V<-V_supply: V=-V_supply
        i = (V-(ke)*w*G)/R
        
        return V,i


In [ ]:
xml_template = """
<mujoco>
  <option timestep="{timestep:e}"/>
  <option>
     <flag gravity="enable" contact="enable" />
  </option>
  <compiler angle="degree" />
    <visual><global offwidth="{width}" offheight="{height}" /></visual>    

  <worldbody>
    <light name="top" pos="0 0 1"/>
    <camera name="world" mode="fixed" pos="0 -.5 .05" axisangle="1 0 0 90"/>
    <camera name="target" mode="targetbody" target="trunk" pos="0 -.5 .05" axisangle="1 0 0 90" />

    
    <body name="floor" pos="0 0 0">
      <geom name="floor" pos="0 0 0" size="1 1 .1" type="plane" rgba="1 0.83 0.61 0.5"/>
    </body>

    <body name="trunk" pos="0 0 .1">
        <joint type="free"/>
        <geom type="box" size=".1 .05 .003" pos="0 0 0" rgba="1 0 0 1" mass=".05"/>
        <geom type="box" size=".005 .005 .05" pos=".1 .05 -.05" rgba="1 0 0 1"/>
        <geom type="box" size=".005 .005 .05" pos="-.1 .05 -.05" rgba="1 0 0 1"  />
        <geom type="box" size=".005 .005 .05" pos=".1 -.05 -.05" rgba="1 0 0 1"/>
        <geom type="box" size=".005 .005 .05" pos="-.1 -.05 -.05" rgba="1 0 0 1"  />
        

      <body name="leg1_l1" pos=".1 0 0" axisangle="0 1 0 70">
          <joint name="leg1_j1" type="hinge" axis="0 1 0" pos="0 0 0" stiffness="{k:e}" damping="{b:e}" limited="false" range="-179 179"/>
          <geom type="box" size=".03 .01 .001" pos=".03 0 0" rgba="1 0 0 1"/>
          <body name="leg1_l2" pos=".06 0 0">
            <joint name="leg1_j2" type="hinge" axis="0 1 0"  stiffness="{k:e}" damping="{b:e}"  limited="false" range="-179 179"/>
            <geom type="box" size=".03 .01 .001" pos=".03 0 0" rgba="1 0 0 1"/>
            <body name="leg1_l4_2" pos=".06 0 0">
              <joint name="leg1_j3" type="hinge" axis="0 1 0"  stiffness="{k:e}" damping="{b:e}"  limited="false" range="-179 179"/>
              <geom type="box" size=".03 .01 .001" pos=".03 0 0" rgba="1 0 0 1"/>
            </body>
          </body>
      </body>

      <body name="leg1_l3" pos=".05 0 0" axisangle="0 1 0 110">
          <joint name="leg1_j4" type="hinge" axis="0 1 0" pos="0 0 0" stiffness="{k:e}" damping="{b:e}" limited="false" range="-179 179"/>
          <geom type="box" size=".03 .01 .001" pos=".03 0 0" rgba="1 0 0 1"/>
          <body name="leg1_l4" pos=".06 0 0">
            <joint name="leg1_j5" type="hinge" axis="0 1 0"   stiffness="{k:e}" damping="{b:e}" limited="false" range="-179 179"/>
            <geom type="box" size=".03 .01 .001" pos=".03 0 0" rgba="1 0 0 1"/>
          </body>
      </body>


    </body>
  </worldbody>

   <contact>
    <exclude body1="trunk" body2="leg1_l1" />
    <exclude body1="trunk" body2="leg1_l2" />
    <exclude body1="trunk" body2="leg1_l3" />
    <exclude body1="trunk" body2="leg1_l4" />
    <exclude body1="trunk" body2="leg1_l4_2" />
    <exclude body1="leg1_l1" body2="leg1_l2" />
    <exclude body1="leg1_l1" body2="leg1_l3" />
    <exclude body1="leg1_l1" body2="leg1_l4" />
    <exclude body1="leg1_l1" body2="leg1_l4_2" />
    <exclude body1="leg1_l2" body2="leg1_l3" />
    <exclude body1="leg1_l2" body2="leg1_l4" />
    <exclude body1="leg1_l2" body2="leg1_l4_2" />
    <exclude body1="leg1_l3" body2="leg1_l4" />
    <exclude body1="leg1_l3" body2="leg1_l4_2" />
    <exclude body1="leg1_l4" body2="leg1_l4_2" />
  </contact>

  
  <actuator>
    <motor name="m1" joint="leg1_j1"/>
    <motor name="m2" joint="leg1_j4"/>
  </actuator>

  <equality>
    <weld name="weld1" active="true" body1="leg1_l4_2" body2="leg1_l4" relpose=".03 0 0 0 0 1 0" anchor=".03 0 0" solimp=".90 .99 .01" solref=".001 1"/>
  </equality>  

  <sensor>
  <framequat objtype="body" name="trunkquat" objname="trunk" />
  <framezaxis objtype="body" name="trunkzaxis" objname="trunk" />
  </sensor>
  <!--
  -->
</mujoco>
"""

xml = xml_template.format(width=width,height=height,k=1e-3,b=1e-5,timestep=timestep)


In [ ]:
model = mujoco.MjModel.from_xml_string(xml)
data = mujoco.MjData(model)
renderer = mujoco.Renderer(model,width=width,height=height)


duration = 5  # (seconds)
framerate = 30  # (Hz)


A0 = 25 * math.pi/180
f = 1
t0_1 = 0
t0_2 = .25
# b1 = 70*math.pi/180
# b2 = 110*math.pi/180
b1 = 0
b2 = 0

def my_controller(model, data):
    t = data.time
    if t<1:
      A=0
    elif t<2:
      A = A0*(t-1)/(2-1)
    else:
      A = A0
    # A=0
    j1 = A*math.sin(2*math.pi*(f*t-t0_1))+b1
    j2 = A*math.sin(2*math.pi*(f*t-t0_2))+b2

    qactual1 = data.joint('leg1_j1').qpos[0]
    wactual1 = data.joint('leg1_j1').qvel[0]
    torque1,V1,i1 = SG90Data.calc_servo_torque(j1,qactual1,wactual1)


    qactual2 = data.joint('leg1_j4').qpos[0]
    wactual2 = data.joint('leg1_j4').qvel[0]
    torque2,V2,i2 = SG90Data.calc_servo_torque(j2,qactual2,wactual2)


    data.ctrl[0] = torque1
    data.ctrl[1] = torque2

    return


try:
    # mujoco.set_mjcb_control(my_controller)
    frames = []
    mujoco.mj_resetData(model, data)
    data.qpos[:] =numpy.array([0.00025277742758359335,
 -2.726436521643972e-18,
 0.0999713489522115,
 0.9999999999299372,
 -3.405554257737078e-19,
 1.1837517309791702e-05,
 -3.083455692201877e-18,
 0.012207735012490992,
 1.1796765078888658,
 1.454498006761503,
 -0.013237197784641433,
 -1.1782036422361484])

    xys = []
    cs = []
    c2s = []
    positive_factors = []
    negative_factors = []
    power = []
    qs = []
    efficiencies = []
    energy_in = []
    energy_out = []

    while data.time < duration:
        xy = data.body('trunk').xpos.copy()
        x_position_before = xy[0]
        last_time = data.time

        my_controller(model,data)
        action = data.ctrl
        j1,j2 = action.flatten()

        qactual1 = data.joint('leg1_j1').qpos[0]
        wactual1 = data.joint('leg1_j1').qvel[0]
        torque1,V1,i1 = SG90Data.calc_servo_torque(j1,qactual1,wactual1)


        qactual2 = data.joint('leg1_j4').qpos[0]
        wactual2 = data.joint('leg1_j4').qvel[0]
        torque2,V2,i2 = SG90Data.calc_servo_torque(j2,qactual2,wactual2)

        pm1 = abs(torque1*wactual1)
        pe1 = abs(V1*i1)

        pm2 = abs(torque2*wactual2)
        pe2 = abs(V2*i2)
        power.append([pe1,pm1,pe2,pm2])
        efficiency = (pm1+pm2)/(pe1+pe2)

        efficiencies.append(efficiency)
        
        mujoco.mj_step(model, data)

        qs.append(data.qpos.copy())

        xy = data.body('trunk').xpos.copy()
        xys.append(xy)
        q = data.body('trunk').xquat.copy()
        q = Quaternion(*q)
        R1 = numpy.array(q.to_rot(),dtype=numpy.float64)
        v = R1@numpy.array([[0,0,1]]).T
        z_dots = ((numpy.array([[0,0,1]])@v).squeeze())
        if z_dots>1:
           z_dots = 1
        tip_angle = numpy.arccos(z_dots)*180/math.pi

        q1 = Quaternion(*data.body('leg1_l4').xquat)
        q2 = Quaternion(*data.body('leg1_l4_2').xquat)

        v1 = numpy.array(q1.to_rot(),dtype=numpy.float64)@numpy.array([[0,0,1]]).T
        v2 = numpy.array(q2.to_rot(),dtype=numpy.float64)@numpy.array([[0,0,-1]]).T

        time = data.time
        constraint_vector_dot = (v1.T@v2).squeeze()
        if constraint_vector_dot>1:
           constraint_vector_dot = 1
        constraint_violation_angle = numpy.arccos(constraint_vector_dot)*180/math.pi

        z_violation = (((xy[2]-.1)**2)**.5)

        control_cost = ((action.flatten())**2).sum()

        forward_progress = xy[0]

        # reward = forward_progress*10 - z_violation*.1 - c*.1 - control_cost*.1 - constraint_violation_angle*.1
        current_velocity = (forward_progress - x_position_before)/(time  - last_time)

        positive_factors.append((forward_progress,time,current_velocity))
        negative_factors.append((z_violation, tip_angle, control_cost, constraint_violation_angle))

        if len(frames) < data.time * framerate:
            renderer.update_scene(data,camera='target')
            pixels = renderer.render()
            frames.append(pixels)

except Exception as ex:
    mujoco.set_mjcb_control(None)
    raise

mujoco.set_mjcb_control(None)

xys = numpy.array(xys)
qs = numpy.array(qs)
positive_factors = numpy.array(positive_factors)
negative_factors = numpy.array(negative_factors)
efficiencies = numpy.array(efficiencies)
power = numpy.array(power)

# Simulate and display video.
media.show_video(frames, fps=framerate,width=width,height=height)

In [ ]:
data.qpos*180/math.pi

In [ ]:
import matplotlib.pyplot as plt
plt.plot(xys)
# xy.shape

In [ ]:
plt.plot(power[:,(0)])

In [ ]:
plt.plot(power[:,(2,3)])

In [ ]:
m = model.body('trunk').mass[0]
g = 9.81

In [ ]:
mechanical_work = m*g*xys[:,0]
plt.plot(mechanical_work)

In [ ]:
import scipy.integrate as si
electrical_power = power[:,(0,2)].sum(1)
electrical_energy = si.cumulative_trapezoid(electrical_power,dx = timestep,initial=0)

plt.plot(electrical_energy)



In [ ]:
COT = mechanical_work/electrical_energy
plt.plot(COT[200:])

In [ ]:
plt.plot(efficiencies)

In [ ]:
qs[500].flatten().tolist()